In [1]:
import requests
from bs4 import BeautifulSoup
import username_n_password

In [2]:
# Gửi request có headers với proxy + authenticate đến web Và xử lý pagination

# fake user-agent để tránh bị chặn
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'}

# đọc danh sách proxy từ file proxy_list.txt và loại bỏ dòng trống
with open('proxy_list.txt', 'r') as p:
    proxy_list = [proxy.strip() for proxy in p if proxy.strip()] # list comprehension loại bỏ khoảng trắng, tab (\t), hoặc dấu xuống dòng (\n) ở đầu và cuối chuỗi.

# chỉ định URL trang web đích
url = "https://quotes.toscrape.com/"

# lặp qua từng proxy trong file để gửi request
for proxy in proxy_list:
    username = username_n_password.username  # Lấy username từ file 'username_password.py' 
    password = username_n_password.password  # Lấy password từ file 'username_password.py' 
    
    proxy_url = f"http://{username}:{password}@{proxy}"  # Tạo URL của proxy + authenticate
    proxies = {
        "http://" : proxy_url,
        "https://" : proxy_url
    }

    # thiết lập try - except để kiểm tra lỗi requests
    try:
        response = requests.get(url, proxies = proxies, headers = headers, timeout = 10)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        response.raise_for_status() # Nếu trang web đích gặp lỗi HTTP (403, 500,..) thì báo lỗi
        
        if response.status_code == 200:
            print(f'Requests thành công. Proxy {proxy}')
        else:
            print(f'Requests thất bại. Proxy: {proxy} - Status code: {response.status_code}')
    except requests.exceptions.ProxyError: # Xử lý lỗi proxy không hoạt động
        print(f"Proxy lỗi: {proxy}. Thử proxy khác...")
    except requests.exceptions.ConnectTimeout: # Xử lý lỗi timeout khi proxy quá chậm
        print(f"Proxy quá chậm: {proxy}. Bỏ qua...")
    except requests.exceptions.ConnectionError: # Xử lý lỗi khi proxy không thể kết nối đến server
        print(f"Không thể kết nối: {proxy}. Thử proxy khác...")
    except requests.exceptions.HTTPError as err: # Xử lý lỗi HTTP khác (403, 404, 500,...)
        print(f"HTTP Error: {err}")
    except Exception as e: # Xử lý các lỗi khác không xác định
        print(f"Lỗi khác: {e}")

Requests thành công. Proxy 113.160.133.32:8080


    Print out the first quote

In [3]:
quote_first = soup.find('div', class_='quote').span.text
print(quote_first)

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”


    Print out the first author

In [4]:
author_first = soup.find('small', class_='author').text
print(author_first)

Albert Einstein


    Print out the first tags

In [5]:
# Tìm div với class 'tags'
tags_div = soup.find('div', class_='tags')

# Tìm tất cả thẻ <a> bên trong div
tags = tags_div.find_all('a', class_='tag')

# Iterate và in ra thông tin của từng tag
for tag in tags:
    tag_text = tag.text.strip()  # Lấy nội dung của tag
    print(tag_text)

change
deep-thoughts
thinking
world


    Take the list of all quote elements on the first page

In [6]:
# Khởi tạo danh sách rỗng chứa data về element của các quote
quote_data = []

# Tìm vị trí chứa tất cả các quote trong first page
quote_divs = soup.find_all('div', class_='quote')

# Chạy vòng lặp cho từng div nhỏ chứa data về các element của quote (nội dung quote, tác giả, danh sách tags)
for quote_div in quote_divs:
    # Lấy nội dung quote (chỉ cần find trong quote_div nhỏ để lấy giá trị khớp đầu tiên với lần lặp này)
    quote_text = quote_div.find('span', class_='text').text

    # Lấy tên tác giả'
    author_text = quote_div.find('small', class_='author').text

    # Lấy danh sách tags
    tags_div = quote_div.find('div', class_='tags')
    # Tìm tất cả thẻ <a> bên trong div
    tags = tags_div.find_all('a', class_='tag')
    # Khởi tạo danh sách tags rỗng
    list_of_tags = []
    # Iterate và in ra thông tin của từng tag
    for tag in tags:
        tag_text = tag.text.strip()  # Lấy nội dung của tag
        list_of_tags.append(tag_text)

    # Lưu data của tất cả element lấy được từ vòng lặp lần này vào danh sách quote_data
    quote_data.append({'quote':quote_text, 'author':author_text, 'tags':list_of_tags})

    Print out quote, author and tags of all elements on the first page (You must use a loop)

In [7]:
# In kết quả
for quote in quote_data:
    print(f"Quote: {quote['quote']}\nAuthor: {quote['author']}\nTags: {', '.join(quote['tags'])}")
    print('-' * 60)

Quote: “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Author: Albert Einstein
Tags: change, deep-thoughts, thinking, world
------------------------------------------------------------
Quote: “It is our choices, Harry, that show what we truly are, far more than our abilities.”
Author: J.K. Rowling
Tags: abilities, choices
------------------------------------------------------------
Quote: “There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Author: Albert Einstein
Tags: inspirational, life, live, miracle, miracles
------------------------------------------------------------
Quote: “The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Author: Jane Austen
Tags: aliteracy, books, classic, humor
------------------------------------------------------------
Quote: “Imperfection is beauty, madness 

    Print out the first author on every page (There are 10 pages)

In [8]:
# xử lý pagination với button 'next'

url = 'https://quotes.toscrape.com/page/1/' # => Khai báo url gốc là page 1 cho lần đầu tiên

page_no = 1 # mặc định sẽ bắt đầu chạy từ page đầu tiên

while True:
    print(f'Scraping page {page_no}')
    response = requests.get(url, proxies = proxies, headers = headers, timeout = 10)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Tìm fisrt author của mỗi page
    page_first_author = soup.find('small', class_='author').text
    print(page_first_author)

    # Kiểm tra và cập nhật 'next_btn'
    next_btn = soup.find('li', class_='next')
    if next_btn:
        next_page_url = next_btn.find('a')['href']
        url = f"https://quotes.toscrape.com{next_page_url}" # => Cập nhật url mới sau khi hoàn thành mỗi vòng lặp
        page_no += 1
    else:
        # Khi không còn button 'next', kết thúc vòng lặp
        print("Hoàn thành scraping tất cả các trang!")
        break

Scraping page 1
Albert Einstein
Scraping page 2
Marilyn Monroe
Scraping page 3
Pablo Neruda
Scraping page 4
Dr. Seuss
Scraping page 5
George R.R. Martin
Scraping page 6
Jane Austen
Scraping page 7
Charles Bukowski
Scraping page 8
Alfred Tennyson
Scraping page 9
Albert Einstein
Scraping page 10
J.K. Rowling
Hoàn thành scraping tất cả các trang!


    Locate the 'Next' button on page 1 and print out its 'href' attribute

In [9]:
url = 'https://quotes.toscrape.com/page/1/'
response = requests.get(url, proxies = proxies, headers = headers, timeout = 10)
soup = BeautifulSoup(response.text, 'html.parser')
next_btn = soup.find('li', class_='next')
next_btn_href = next_btn.find('a')['href']
print(next_btn_href)

/page/2/


    Using the 'Next' button, reach to the last quote element in page one (use parents/siblings)

In [10]:
last_quote_element = next_btn.parent.parent.find_previous_sibling()
print(last_quote_element)

<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“A day without sunshine is like, you know, night.”</span>
<span>by <small class="author" itemprop="author">Steve Martin</small>
<a href="/author/Steve-Martin">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="humor,obvious,simile" itemprop="keywords"/>
<a class="tag" href="/tag/humor/page/1/">humor</a>
<a class="tag" href="/tag/obvious/page/1/">obvious</a>
<a class="tag" href="/tag/simile/page/1/">simile</a>
</div>
</div>


    Print out the class name of the last element

In [11]:
last_quote_element = next_btn.parent.parent.find_previous_sibling()
last_element_class = last_quote_element['class']
print(last_element_class)

['quote']


    Print out the itemtype of the last element

In [12]:
last_quote_element = next_btn.parent.parent.find_previous_sibling()
last_element_class = last_quote_element['itemtype']
print(last_element_class)

http://schema.org/CreativeWork
